<a href="https://colab.research.google.com/github/fitristachan/Air-Quality-Data-Analytic/blob/main/Air_Quality_Data_Analytic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: Air Quality Dataset
- **Nama:** Fitri Sagita
- **Email:** fitristarius@gmail.com
- **ID Dicoding:** fitristachan

## Menentukan Pertanyaan Bisnis


- Bagaimana perkembangan parameter kualitas udara di beberapa stasiun pengukuran di Tiongkok selama periode 2013-2017?
- Pada rentang waktu manakah (pagi, siang, sore, atau malam) rata-rata nilai parameter kualitas udara menunjukkan angka tertinggi di Tiongkok selama periode 2013-2017?

## Import Semua Packages/Library yang Digunakan

In [1]:
import os
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
import scipy as py
import seaborn as sns

## Data Wrangling

### Gathering Data

Karena ada lebih dari satu file dataset dalam folder, maka kita looping terlebih dahulu untuk mendapatkan semua pathfile. Tujuannya agar lebih mudah. kemudian muat tiap file menggunakan .read_csv dan masukkan dataframenya ke dalam list airquality_df_list. Karena tiap tabel memiliki field yang sama dan sudah ada pembedanya di station sebagai penanda darimana data tersebut berasal, maka kita lakukan concat untuk menjadikan semua tabel jadi satu dataframe.

In [2]:

directory = os.fsencode("D:\Chase\Certification-Submission\Air-quality-dataset\Air-Quality-Data-Analytic\PRSA_Data")
airquality_df_list = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        if isinstance(directory, bytes):
            directory = directory.decode('utf-8')
        data_path = os.path.join(directory, filename)
        df = pd.read_csv(data_path)
        airquality_df_list.append(df)
    else:
        print("Gagal menngambil path file")

airquality_df = pd.concat(airquality_df_list, ignore_index=True)
print(airquality_df)

           No  year  month  day  hour  PM2.5  PM10   SO2   NO2     CO    O3  \
0           1  2013      3    1     0    4.0   4.0   4.0   7.0  300.0  77.0   
1           2  2013      3    1     1    8.0   8.0   4.0   7.0  300.0  77.0   
2           3  2013      3    1     2    7.0   7.0   5.0  10.0  300.0  73.0   
3           4  2013      3    1     3    6.0   6.0  11.0  11.0  300.0  72.0   
4           5  2013      3    1     4    3.0   3.0  12.0  12.0  300.0  72.0   
...       ...   ...    ...  ...   ...    ...   ...   ...   ...    ...   ...   
420763  35060  2017      2   28    19   11.0  32.0   3.0  24.0  400.0  72.0   
420764  35061  2017      2   28    20   13.0  32.0   3.0  41.0  500.0  50.0   
420765  35062  2017      2   28    21   14.0  28.0   4.0  38.0  500.0  54.0   
420766  35063  2017      2   28    22   12.0  23.0   4.0  30.0  400.0  59.0   
420767  35064  2017      2   28    23   13.0  19.0   4.0  38.0  600.0  49.0   

        TEMP    PRES  DEWP  RAIN   wd  WSPM        

**Insight:**
- Dataset yang ada berjumlah 420768 dimulai dari index 0 - 420767
- Temperature ada yang minus dan ada yang plus
- Data yang ada berasal dari tahun 2013 - 2017
- Terdapat 18 kolom, dengan 11 diantaranya merupakan hasil ukur komponen dan partikel yang biasanya menjadi indikator dalam penilaian kualitas udara
- Semua hasil ukur bertipe data float

### Assessing Data

Lihat isi air quality

In [3]:
airquality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    412029 non-null  float64
 6   PM10     414319 non-null  float64
 7   SO2      411747 non-null  float64
 8   NO2      408652 non-null  float64
 9   CO       400067 non-null  float64
 10  O3       407491 non-null  float64
 11  TEMP     420370 non-null  float64
 12  PRES     420375 non-null  float64
 13  DEWP     420365 non-null  float64
 14  RAIN     420378 non-null  float64
 15  wd       418946 non-null  object 
 16  WSPM     420450 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB


Lihat berapa missing value di tiap field dan mengecek ada atau tidaknya duplikasi data

In [4]:
airquality_df.isna().sum()

No             0
year           0
month          0
day            0
hour           0
PM2.5       8739
PM10        6449
SO2         9021
NO2        12116
CO         20701
O3         13277
TEMP         398
PRES         393
DEWP         403
RAIN         390
wd          1822
WSPM         318
station        0
dtype: int64

In [5]:
print("Jumlah duplikasi: ", airquality_df.duplicated().sum())

Jumlah duplikasi:  0


Kode di bawah untuk melihat parameter statistik data dan karena nilai max dalam tiap indikator terlalu besar, sedangkan saya bukan expertise dalam penilaian indikator cuaca dan sumber yang ada di internet pun terbatas, sehingga saya putuskan untuk melakukan grouping nilai maks berdasarkan tiap station untuk dilihat dimanakah nilai maks itu dan apakah ada perbedaan yang sangat signifikan dibanding nilai maks pada station yang lain atau tidak.

In [6]:
airquality_df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,412029.000000,414319.000000,411747.000000,408652.000000,400067.000000,407491.000000,420370.000000,420375.000000,420365.000000,420378.000000,420450.000000
mean,17532.500000,2014.662560,6.522930,15.729637,11.500000,79.793428,104.602618,15.830835,50.638586,1230.766454,57.372271,13.538976,1010.746982,2.490822,0.064476,1.729711
std,10122.116943,1.177198,3.448707,8.800102,6.922195,80.822391,91.772426,21.650603,35.127912,1160.182716,56.661607,11.436139,10.474055,13.793847,0.821004,1.246386
min,1.000000,2013.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.026500,100.000000,0.214200,-19.900000,982.400000,-43.400000,0.000000,0.000000
25%,8766.750000,2014.000000,4.000000,8.000000,5.750000,20.000000,36.000000,3.000000,23.000000,500.000000,11.000000,3.100000,1002.300000,-8.900000,0.000000,0.900000
50%,17532.500000,2015.000000,7.000000,16.000000,11.500000,55.000000,82.000000,7.000000,43.000000,900.000000,45.000000,14.500000,1010.400000,3.100000,0.000000,1.400000
75%,26298.250000,2016.000000,10.000000,23.000000,17.250000,111.000000,145.000000,20.000000,71.000000,1500.000000,82.000000,23.300000,1019.000000,15.100000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,999.000000,999.000000,500.000000,290.000000,10000.000000,1071.000000,41.600000,1042.800000,29.100000,72.500000,13.200000


In [7]:
max_per_station = airquality_df.groupby('station').max(numeric_only=True)
print(max_per_station)

                  No  year  month  day  hour  PM2.5   PM10    SO2    NO2  \
station                                                                    
Aotizhongxin   35064  2017     12   31    23  898.0  984.0  341.0  290.0   
Changping      35064  2017     12   31    23  882.0  999.0  310.0  226.0   
Dingling       35064  2017     12   31    23  881.0  905.0  156.0  205.0   
Dongsi         35064  2017     12   31    23  737.0  955.0  300.0  258.0   
Guanyuan       35064  2017     12   31    23  680.0  999.0  293.0  270.0   
Gucheng        35064  2017     12   31    23  770.0  994.0  500.0  276.0   
Huairou        35064  2017     12   31    23  762.0  993.0  315.0  231.0   
Nongzhanguan   35064  2017     12   31    23  844.0  995.0  257.0  273.0   
Shunyi         35064  2017     12   31    23  941.0  999.0  239.0  258.0   
Tiantan        35064  2017     12   31    23  821.0  988.0  273.0  241.0   
Wanliu         35064  2017     12   31    23  957.0  951.0  282.0  264.0   
Wanshouxigon

**Insight:**
- Terdapat banyak missing value di PM2.5, PM10, SO2, NO2, CO, O3, TEMP, PRES, DEWP, RAIN, wd, dan WSPM
- Tidak ada duplikasi data
- Max Nomor tidak sesuai jumlah data sehingga perlu dilakukan penomoran ulang
- SO2 wajar karena dari jurnal yang dirilis tahun 2020 dan berjudul "VERIFIKASI METODE PENENTUAN KADAR SULFUR
DIOKSIDA (SO2) DALAM UDARA AMBIEN SECARA
SPEKTROFOTOMETRI UV-VISIBEL DI PT. KARSA BUANA
LESTARI" oleh DT Metia, link: https://dspace.uii.ac.id/bitstream/handle/123456789/28492/17231059%20Dinda%20Tantri%20Metia.pdf?sequence=1, nilai maksnya adalah 900
- Dilihat dari grouping nilai max yang tidak wajar adalah nilai O3 di Dongsi


### Cleaning Data

Mengganti nilai max dongsi yang abnormal menjadi nilai rata-rata di jam yang sama. Karena setelah dicek ditabel ada beberapa nilai max abnormal yang lebih dari 1000, maka kita tetapkan nilai abnormalnya adalah > 1000.

In [8]:
def replace_dongsi_abnormal(row):
    if row['O3'] >= 1000:
        time_data = airquality_df[(airquality_df['station'] == 'Dongsi') &
                                (airquality_df['month'] == row['month']) & 
                                (airquality_df['day'] == row['day']) & 
                                (airquality_df['hour'] == row['hour'])]
        dongsi_mean = time_data['O3'].mean()
        return dongsi_mean
    else:
        return row['O3']

airquality_df['O3'] = airquality_df.apply(replace_dongsi_abnormal, axis=1)

max_per_station = airquality_df.groupby('station').max(numeric_only=True)
print(max_per_station)

                  No  year  month  day  hour  PM2.5   PM10    SO2    NO2  \
station                                                                    
Aotizhongxin   35064  2017     12   31    23  898.0  984.0  341.0  290.0   
Changping      35064  2017     12   31    23  882.0  999.0  310.0  226.0   
Dingling       35064  2017     12   31    23  881.0  905.0  156.0  205.0   
Dongsi         35064  2017     12   31    23  737.0  955.0  300.0  258.0   
Guanyuan       35064  2017     12   31    23  680.0  999.0  293.0  270.0   
Gucheng        35064  2017     12   31    23  770.0  994.0  500.0  276.0   
Huairou        35064  2017     12   31    23  762.0  993.0  315.0  231.0   
Nongzhanguan   35064  2017     12   31    23  844.0  995.0  257.0  273.0   
Shunyi         35064  2017     12   31    23  941.0  999.0  239.0  258.0   
Tiantan        35064  2017     12   31    23  821.0  988.0  273.0  241.0   
Wanliu         35064  2017     12   31    23  957.0  951.0  282.0  264.0   
Wanshouxigon

Karena dari proses assesing diketahui banyak sekali missing value dan tidak mungkin untuk menghapus semua data yang mengandung missing value karena akan menyebabkan kualitas data menurun, maka missing value tersebut akan diganti dengan menggunakan teknik imputation. Saya menggunakan mean dari tiap group, kecuali wd. Menggunakan mean dari tiap group tujuannya agar nilai pengganti lebih akurat karena merupakan nilai rata-rata dari tiap station.

In [9]:
list_missing = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
for column in list_missing:
    mean_per_column = airquality_df.groupby('station')[column].mean()
    airquality_df[column] = airquality_df.apply(
        lambda row: mean_per_column[row['station']] if pd.isna(row[column]) else row[column],
        axis=1
)

modus_wd_per_station = airquality_df.groupby('station')['wd'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
airquality_df['wd'] = airquality_df.apply(
    lambda row: modus_wd_per_station[row['station']] if pd.isna(row['wd']) else row['wd'],
    axis=1
)

airquality_df.isna().sum()

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

Mengubah kolom nomor yang masih berantakan menjadi berurutan sesuai jumlah data

In [10]:
airquality_df['No'] = airquality_df.index + 1
airquality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    420768 non-null  float64
 6   PM10     420768 non-null  float64
 7   SO2      420768 non-null  float64
 8   NO2      420768 non-null  float64
 9   CO       420768 non-null  float64
 10  O3       420768 non-null  float64
 11  TEMP     420768 non-null  float64
 12  PRES     420768 non-null  float64
 13  DEWP     420768 non-null  float64
 14  RAIN     420768 non-null  float64
 15  wd       420768 non-null  object 
 16  WSPM     420768 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB


**Insight:**
- Untuk mengisi missing value secara spesifik, di kolom berapa, tidak dapat menggunakan fungsi .fillna. Sehingga saya menggunakan apply, karena datanya tidak mau terisi jika menggunakan .fillna.
- Untuk menemukan modus secara spesifik dari kolom tertentu dengan grouping stasiun juga tidak dapat menggunakan .mode secara langsung dan harus menggunakan aggregate dengan fungsi yang dimau didalamnya.
- Dilakukan replace data abnormal terlebih dahulu baru mengisi missing value karena data abnormal yang ada dapat memengaruhi pengisian missing value dikarenakan nilai valuenya terlalu besar sehingga dapat membuat nilai mean yang dicari membesar, sehingga diketahui urutan cleaning data harus didasarkan pada kebutuhan data
- Missing value dan data abnormal tidak dihapus dengan pertimbangan pertanyaan yang ingin dijawab berhubungan erat dengan waktu yang ada sehingga jika dihapus maka dapat menyebabkan hilangnya nilai di waktu tertentu yang menyebabkan ketidakakuratan hasil.

## Exploratory Data Analysis (EDA)

### Explore ...

In [11]:
airquality_df.describe(include="all")

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
count,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768.00000,420768.000000,420768.000000,420768.000000,420768.000000,420768.000000,420768,420768.000000,420768
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,12
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NE,NaN,Aotizhongxin
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43699,NaN,35064
mean,210384.500000,2014.662560,6.522930,15.729637,11.500000,79.780235,104.578547,15.820683,50.583319,1230.62509,57.321803,13.538931,1010.746341,2.490654,0.064475,NaN,1.729701,NaN
std,121465.403371,1.177198,3.448707,8.800102,6.922195,79.984694,91.075152,21.420157,34.677400,1131.69545,55.450720,11.430737,10.469404,13.787251,0.820624,NaN,1.245924,NaN
min,1.000000,2013.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.285600,1.026500,100.00000,0.214200,-19.900000,982.400000,-43.400000,0.000000,NaN,0.000000,NaN
25%,105192.750000,2014.000000,4.000000,8.000000,5.750000,21.000000,36.000000,3.000000,24.000000,500.00000,12.000000,3.100000,1002.300000,-8.900000,0.000000,NaN,0.900000,NaN
50%,210384.500000,2015.000000,7.000000,16.000000,11.500000,57.000000,83.739723,8.000000,44.000000,900.00000,47.000000,14.500000,1010.400000,3.000000,0.000000,NaN,1.400000,NaN
75%,315576.250000,2016.000000,10.000000,23.000000,17.250000,109.000000,144.000000,19.000000,70.000000,1500.00000,80.000000,23.200000,1019.000000,15.100000,0.000000,NaN,2.200000,NaN


In [12]:
airquality_df.sort_values(by=["year", "station"], ascending=True)

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420763,420764,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,NW,2.4,Wanshouxigong
420764,420765,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Wanshouxigong
420765,420766,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,NW,1.1,Wanshouxigong
420766,420767,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Wanshouxigong


In [13]:
airquality_df.groupby(by=["year", "station"]).agg({
    "PM2.5":  ["min", "max", "mean"],
    "PM10":  ["min", "max", "mean"],
    "SO2" :  ["min", "max", "mean"],
    "NO2" :  ["min", "max", "mean"],
    "CO" :  ["min", "max", "mean"],
    "O3" :  ["min", "max", "mean"],
    "TEMP" :  ["min", "max", "mean"],
    "PRES" :  ["min", "max", "mean"],
    "DEWP":  ["min", "max", "mean"],
    "RAIN":  ["min", "max", "mean"],
    "wd" : [lambda x: x.mode().iloc[0] if not x.mode().empty else None],
    "WSPM":  ["min", "max", "mean"]
})

PM2.5                    PM10                        SO2  \
                     min    max        mean  min    max        mean     min   
year station                                                                  
2013 Aotizhongxin    3.0  665.0   82.393043  2.0  844.0  112.913167  0.2856   
     Changping       2.0  443.0   72.637567  2.0  653.0   91.257159  0.2856   
     Dingling        3.0  410.0   64.777450  2.0  720.0   78.763031  0.2856   
     Dongsi          3.0  520.0   86.925760  2.0  602.0  105.018146  0.2856   
     Guanyuan        3.0  512.0   82.140551  2.0  999.0  107.772775  1.0000   
     Gucheng         2.0  500.0   82.035115  2.0  983.0  112.600537  0.2856   
     Huairou         2.0  534.0   68.715264  2.0  887.0   94.680239  0.2856   
     Nongzhanguan    3.0  844.0   84.350177  2.0  811.0  103.845434  0.5712   
     Shunyi          3.0  558.0   77.280065  2.0  764.0   94.662433  0.2856   
     Tiantan         3.0  525.0   83.197686  2.0  839.0  108.253070  0.5712   
     Wanliu          3.0  560.0   91.572150  2.0  829.0  112.282788  0.2856   
     Wanshouxigong   3.0  510.0   84.033836  2.0  961.0  111.315051  0.2856   
2014 Aotizhongxin    3.0  584.0   90.333581  3.0  948.0  121.389921  2.0000   
     Changping       3.0  581.0   79.144216  3.0  999.0  106.318472  1.0000   
     Dingling        3.0  548.0   73.949575  3.0  895.0   90.358112  1.0000   
     Dongsi          3.0  737.0   87.455485  3.0  794.0  116.939118  2.0000   
     Guanyuan        3.0  603.0   88.255731  3.0  814.0  115.736635  1.0000   
     Gucheng         3.0  770.0   89.139820  3.0  862.0  133.565039  1.0000   
     Huairou         3.0  528.0   77.011450  3.0  993.0   99.889143  1.0000   
     Nongzhanguan    3.0  809.0   90.185522  3.0  907.0  125.105613  2.0000   
     Shunyi          3.0  592.0   84.472716  3.0  903.0  110.799902  1.0000   
     Tiantan         3.0  541.0   86.433097  3.0  894.0  114.227999  1.0000   
     Wanliu          3.0  589.0   89.410417  3.0  923.0  130.078648  2.0000   
     Wanshouxigong   3.0  704.0   88.980393  3.0  886.0  125.725547  1.0000   
2015 Aotizhongxin    3.0  657.0   82.134578  3.0  984.0  112.035381  2.0000   
     Changping       3.0  500.0   69.672148  3.0  992.0   94.153708  2.0000   
     Dingling        3.0  647.0   64.627529  3.0  905.0   85.350134  2.0000   
     Dongsi          3.0  685.0   87.326436  3.0  816.0  111.840239  2.0000   
     Guanyuan        3.0  680.0   82.418859  3.0  961.0  106.370617  2.0000   
     Gucheng         3.0  741.0   83.631204  3.0  941.0  116.261947  2.0000   
     Huairou         3.0  762.0   70.635958  3.0  991.0   89.425734  2.0000   
     Nongzhanguan    3.0  667.0   85.932337  3.0  899.0  107.457277  2.0000   
     Shunyi          3.0  689.0   81.250064  3.0  999.0   99.689977  2.0000   
     Tiantan         3.0  671.0   82.600319  3.0  887.0  103.048920  2.0000   
     Wanliu          3.0  718.0   80.585911  3.0  951.0  106.780609  2.0000   
     Wanshouxigong   3.0  690.0   84.722278  3.0  931.0  110.966341  2.0000   
2016 Aotizhongxin    3.0  898.0   74.279022  4.0  884.0   94.842749  2.0000   
     Changping       3.0  882.0   61.778503  3.0  933.0   86.422490  2.0000   
     Dingling        3.0  881.0   60.042212  3.0  864.0   80.514580  2.0000   
     Dongsi          3.0  695.0   80.465800  5.0  891.0  103.282334  2.0000   
     Guanyuan        3.0  639.0   77.228560  3.0  906.0  104.738905  2.0000   
     Gucheng         3.0  646.0   77.530704  3.0  986.0  110.793668  2.0000   
     Huairou         3.0  683.0   61.277062  3.0  973.0   82.391200  2.0000   
     Nongzhanguan    3.0  566.0   76.155790  3.0  995.0   97.038275  2.0000   
     Shunyi          3.0  941.0   72.051435  3.0  917.0   87.816802  2.0000   
     Tiantan         3.0  821.0   74.035380  3.0  988.0   97.231196  2.0000   
     Wanliu          3.0  957.0   71.561885  3.0  919.0   92.986175  2.0000   
     Wanshouxigong   3.0  999.0   78.686672  5.0  929.0   99.097157  

In [14]:
airquality_df.groupby(by=["day", "month", "year", "station"]).agg({
    "PM2.5":  ["min", "max", "mean"],
    "PM10":  ["min", "max", "mean"],
    "SO2" :  ["min", "max", "mean"],
    "NO2" :  ["min", "max", "mean"],
    "CO" :  ["min", "max", "mean"],
    "O3" :  ["min", "max", "mean"],
    "TEMP" :  ["min", "max", "mean"],
    "PRES" :  ["min", "max", "mean"],
    "DEWP":  ["min", "max", "mean"],
    "RAIN":  ["min", "max", "mean"],
    "wd" : ["nunique", lambda x: x.mode().iloc[0] if not x.mode().empty else None],
    "WSPM":  ["min", "max", "mean"]
})

PM2.5                      PM10         \
                                min    max        mean    min    max   
day month year station                                                 
1   1     2014 Aotizhongxin    19.0  124.0   48.583333   54.0  594.0   
               Changping       25.0  116.0   38.541667   72.0  161.0   
               Dingling        23.0   76.0   34.291667   59.0  104.0   
               Dongsi          19.0   95.0   56.083333   36.0  160.0   
               Guanyuan        15.0   84.0   48.750000   54.0  209.0   
...                             ...    ...         ...    ...    ...   
31  12    2016 Nongzhanguan   143.0  487.0  334.291667  143.0  562.0   
               Shunyi          93.0  540.0  221.000000   93.0  625.0   
               Tiantan        258.0  470.0  346.208333  291.0  589.0   
               Wanliu         184.0  475.0  291.458333  203.0  529.0   
               Wanshouxigong  311.0  493.0  394.166667  311.0  537.0   

                                           SO2                     NO2  ...  \
                                    mean   min   max       mean    min  ...   
day month year station                                                  ...   
1   1     2014 Aotizhongxin   169.166667  11.0  59.0  27.416667    9.0  ...   
               Changping       95.375000  15.0  82.0  29.458333   20.0  ...   
               Dingling        81.958333  12.0  35.0  19.125000    7.0  ...   
               Dongsi          99.750000  12.0  57.0  30.916667   12.0  ...   
               Guanyuan       121.583333   9.0  60.0  29.250000   30.0  ...   
...                                  ...   ...   ...        ...    ...  ...   
31  12    2016 Nongzhanguan   377.083333  14.0  28.0  21.875000   82.0  ...   
               Shunyi         239.166667  19.0  44.0  28.833333   50.0  ...   
               Tiantan        425.750000   2.0  22.0   7.791667   80.0  ...   
               Wanliu         321.750000   9.0  69.0  29.000000  128.0  ...   
               Wanshouxigong  421.416667   8.0  31.0  16.916667  108.0  ...   

                              DEWP            RAIN                wd  \
                               max       mean  min  max mean nunique   
day month year station                                                 
1   1     2014 Aotizhongxin   -8.5 -10.637500  0.0  0.0  0.0      12   
               Changping     -12.1 -13.720833  0.0  0.0  0.0       7   
               Dingling      -12.1 -13.720833  0.0  0.0  0.0       7   
               Dongsi         -8.4 -12.141667  0.0  0.0  0.0       9   
               Guanyuan       -8.5 -10.637500  0.0  0.0  0.0      12   
...                            ...        ...  ...  ...  ...     ...   
31  12    2016 Nongzhanguan   -5.0  -7.925000  0.0  0.0  0.0       9   
               Shunyi         -5.8  -9.904167  0.0  0.0  0.0      10   
               Tiantan        -5.0  -7.925000  0.0  0.0  0.0       9   
               Wanliu         -5.6  -8.812500  0.0  0.0  0.0       9   
               Wanshouxigong  -5.0  -7.925000  0.0  0.0  0.0       9   

                                        WSPM                 
                             <lambda_0>  min  max      mean  
day month year station                                       
1   1     2014 Aotizhongxin           W  0.1  3.7  1.283333  
               Changping              W  0.6  5.7  3.266667  
               Dingling               W  0.6  5.7  3.266667  
               Dongsi               WNW  0.4  5.6  1.970833  
               Guanyuan               W  0.1  3.7  1.283333  
...                                 ...  ...  ...       ...  
31  12    2016 Nongzhanguan         ENE  0.1  1.4  0.850000  
               Shunyi               NNE  0.0  1.4  0.970833  
               Tiantan              ENE  0.1  1.4  0.850000  
               Wanliu               WSW  0.0  2.0  0.891667  
               Wanshouxigong        ENE  0.1  1.4  0.850000  

[17532 rows x 35 columns]

In [15]:
airquality_df.groupby(by=["RAIN", "station"]).agg({
    "DEWP":  ["mean"],
    "WSPM":  ["mean"]
})

DEWP      WSPM
                        mean      mean
RAIN station                          
0.0  Aotizhongxin   2.656666  1.715554
     Changping      1.017004  1.869218
     Dingling       1.017004  1.869218
     Dongsi         1.969836  1.865548
     Guanyuan       2.656666  1.715554
...                      ...       ...
52.1 Changping     19.900000  2.300000
     Dingling      19.900000  2.300000
72.5 Aotizhongxin  23.000000  3.900000
     Guanyuan      23.000000  3.900000
     Wanliu        23.000000  3.900000

[1474 rows x 2 columns]

**Insight:**
- Dari tahun 2013-2017 terjadi penurunan yang signifikan pada DEWP dan curah hujan (RAIN)  di tahun 2017, sedangkan pada parameter lain tidak terlihat perubahan yang signifikan.
- Karena ada perbedaan signifikan pada DEWP dan RAIN dan nilai min, max dan mean RAIN mirip dengan WSPM, maka dibuat pivot table berdasarkan curah hujan dan lokasi stasiun, untuk mengetahui hubungan DEWP dan WSPM dengan RAIN. Diketahui bahwa semakin tinggi nilai RAIN di setiap statiun, maka nilai WSPM dan DEWP akan meningkat.

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2